In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from boolrank import DualSiglip2Model
from my_processing import paths_to_dataset

# Load data
data_path = "training"
paths = [
    f"data/{data_path}.jsonl",
    "data/TAR_data.jsonl",
    "data/sysrev_conv.jsonl",
]
dataset = paths_to_dataset(
    paths,
    test_only_sources=['TAR', 'sysrev'],
    train_sources=['pubmed-searchrefiner','pubmed-query','raw-jsonl']
)

# Embedding
N = 1000
df = pd.DataFrame({
    "nl": dataset["train"]["nl_query"],
    "bool": dataset["train"]["bool_query"],
    "quality": dataset["train"]["quality"],
    "source": dataset["train"]["source"]
})
df = df.sample(N, random_state=42).reset_index(drop=True)

data/training.jsonl
data/TAR_data.jsonl
data/sysrev_conv.jsonl


Finding similar: 100%|██████████| 40/40 [00:00<00:00, 79967.66it/s]


In [ ]:
model = DualSiglip2Model('BAAI/bge-small-en-v1.5', loss_type="clip")

embeddings = model.encode_text(df["nl"].tolist()).cpu()

tsne = TSNE(n_components=2,
            perplexity=30,
            learning_rate=200,
            n_iter=1000,
            random_state=42,
            init="pca")
coords = tsne.fit_transform(embeddings)

df["x"] = coords[:,0]
df["y"] = coords[:,1]

fig = px.scatter(
    df, x="x", y="y",
    color="source",               # color by data source
    size="quality",               # optionally size by quality score
    hover_data={
        "nl": True,               # show the NL query on hover
        "bool": True,             # show the bool query
        "quality": True,
        "x": False, "y": False
    },
    title=f"t-SNE of {N} Text Embeddings"
)

fig.update_traces(marker=dict(opacity=0.7, line_width=0.5))
fig.update_layout(width=800, height=600)
fig.show()

c:\Users\Simon\miniconda3\envs\RP\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.